In [7]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import *
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from math import *
#import pandas as pd

from scipy import interpolate
from scipy.optimize import curve_fit 

from astropy.io import fits
from astropy.table import Table
from astropy.table import vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic
# import astropy.units as u
import astropy.units as units
import astropy.coordinates as coord
#from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
# #import matplotlib.ticker as mtick


import os
#%matplotlib widget
import seaborn as sns
import os, sys


# from astropy.io import ascii
# from astropy.coordinates import galactocentric_frame_defaults

import sklearn
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

from scipy import interpolate

import random
from scipy import integrate


gcolor = ['c','blue','g','r','orange', 'green','cyan']

In [8]:
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.sps_util as lib
import sys,glob
from pathlib import Path

from ppxf.ppxf import ppxf, robust_sigma

from vorbin.voronoi_2d_binning import voronoi_2d_binning
from plotbin.display_bins import display_bins

In [9]:
c = 299792.458  # spped of light [km/s]

In [10]:
def Data_presol(filename):
    ori_data = fits.open('./../Ori_Data/'+filename)
    flux_file = ori_data[0].data
    data_hdr = ori_data[0].header
    data_wave = data_hdr['CRVAL3']+np.arange(data_hdr['NAXIS3'])*data_hdr['CD3_3']

    return flux_file, data_hdr, data_wave


In [11]:
sps_name = 'emiles'
ppxf_dir = Path(lib.__file__).parent
basename = f"spectra_{sps_name}_9.0.npz"
filename = ppxf_dir / 'sps_models' / basename

In [12]:
FWHM_gal = None   # set this to None to skip templates broadening
sps = lib.sps_lib(filename, 10.823233432636641, FWHM_gal, norm_range=[4800, 5300])

In [13]:
npix, *reg_dim = sps.templates.shape
sps.templates = sps.templates.reshape(npix, -1)

In [14]:
sps.templates /= np.median(sps.templates) # Normalizes stellar templates by a scalar
regul_err = 0.01 # Desired regularization error

In [15]:
def Data_sol(filename, area_z):

    flux_file, data_hdr, data_wave = Data_presol(filename)

    for i in range(flux_file.shape[1]):
        for j in range(flux_file.shape[2]):

            flux_sig = flux_file[:,i,j]
            galaxy,logLam1,velscale = util.log_rebin(np.array([np.min(data_wave), np.max(data_wave)]),flux_sig)
            galaxy = galaxy/np.median(galaxy)

            if i == 0 and j ==0:
                galaxies = np.ndarray(shape= (galaxy.shape[0],flux_file.shape[1],flux_file.shape[2]))
            else:
                galaxies[:,i,j]=galaxy
    
    logLam10 = logLam1[:]
    logLam1 = logLam10[200:-200]


    vazdekis = glob.glob('./../templates/spectra_sun_vega.npz')
    velscale_ratio = 2

    hdu = fits.open(vazdekis[0]) # FPC
    ssp = hdu[0].data # FPC
    h2 = hdu[0].header # FPC
    lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1'] - 1)]) # FPC
    sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio) # FPC
    templates = np.empty((sspNew.size, len(vazdekis))) # FPC

    for j, file in enumerate(vazdekis):
        hdu = fits.open(file)
        ssp = hdu[0].data
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates[:, j] = sspNew/np.median(sspNew)  # Normalizes templates

    vel_s = c * np.log(1 + area_z)
    vel_dis_s = 40 # Set the dis v = 40 km/s

    velfield = np.ndarray(shape=galaxies.shape[1:3])+np.nan
    sigfield = np.ndarray(shape=galaxies.shape[1:3])+np.nan
    tempnum = np.ndarray(shape=galaxies.shape[1:3])

    for i in range(galaxies.shape[1]):
        for j in range(galaxies.shape[2]):
            tmpgalaxy = galaxies[200:-200,i,j]
            noise = np.full_like(tmpgalaxy,0.1)
            if np.count_nonzero(tmpgalaxy) > 50:
                pp = ppxf(templates, tmpgalaxy, noise, velscale, [vel_s, vel_dis_s], plot=False, moments=4, quiet=True,
                      degree=4, velscale_ratio=velscale_ratio)    
                velfield[i,j] = pp.sol[0]
                sigfield[i,j] = pp.sol[1]
                weights = pp.weights
                indwt = np.where(weights == np.max(weights))[0]
                tempnum[i,j] = indwt[0]

    return velfield


In [16]:
Data_sol('VCC0308_stack.fits',0.0055)

OSError: Zip files with multiple members are not supported.